In [1]:
import json
import pandas as pd
import numpy as np
import a1chemy as a1
import a1chemy.indicators as indicators
import a1chemy.data_source as data_source
from tqdm.notebook import tqdm
import random
import time
import pymongo
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

def magnify():
    width = '80px'
    return [dict(selector="th",
                 props=[('max-width', width), ('width', width), ('word-wrap', 'break-word'), ('text-align', 'center')]),
            dict(selector="td",
                 props=[('max-width', width), ('width', width), ('text-align', 'center')]),
            dict(selector="tr",
                 props=[('max-width', width), ('width', width), ('text-align', 'center')])
]

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client=mongo_client)

# sectors data
# sectors_id = 'tradingview_sectors'
sectors_id = 'swsindex'
sectors_list = a1.read_json_from_file('/Users/line/workspace/freedom/A1chemy/data/sw_sector.json')
swsindex_tag_tree = mongo_tags_client.tree(id=sectors_id)
print('swsindex_tag_tree.len({})'.format(len(swsindex_tag_tree.index_map)))
stock_sector_map = {}
sectors = set()


for key, value in swsindex_tag_tree.root.children.items():
    sectors.add(key)
    for id, stock in value.children.items():
        stock_sector_map[stock.values['symbol']] = key
print('stock_sector_map len({})'.format(len(stock_sector_map)))

stocks_tree = mongo_tags_client.tree(id='csi300')
sci_300_sector_map = {}
stocks = []
for k, stock in stocks_tree.root.children.items():
    sector = stock_sector_map[stock.values['symbol']]
    sector_list = sci_300_sector_map.setdefault(sector, [])
    sector_list.append(stock)
    stocks.append(stock.values)


from a1chemy.common.ticks import Ticks
stock_candle_data_map = {}
stock_symbol_list = [stock['symbol'] for stock in stocks]
print(len(stock_symbol_list))
for tick in mongo_ticks_client.find({'symbol':{'$in': stock_symbol_list}}):   
    tick.raw_data['MA20'] = indicators.sma(data=tick.close(), day=20)
    tick.convert_tz(tz='Asia/Shanghai')
    stock_candle_data_map[tick.symbol] = tick

sci_300_sector_count_map = {}
for k, v in sci_300_sector_map.items():
    sci_300_sector_count_map[k] = len(v)

print('\x1b[31m全行业公司\x1b[0m:')
for k, v in sci_300_sector_count_map.items():
    company_name_list = ",".join([stock_candle_data_map[d.values['symbol']].name for d in sci_300_sector_map[k]])
    print("\x1b[31m{}({})\x1b[0m:\n{}".format(k, v, company_name_list))


## 画图
breath_color_list = [
    '#FF0000',
    '#FF3333',
    '#FF6666',
    '#FF9999',
    '#FFCCCC',
    '#CCFFCC',
    '#99FF99',
    '#66FF66',
    '#33FF33',
    '#00CC00'
]

def market_breath_color(v):
    '''
    highlight the maximum in a Series yellow.
    '''
    color_index = 0 if float(v) <= 0 else int(float(v)/10-1)
    return 'background-color: ' + breath_color_list[color_index]


start_time = pd.Timestamp("2020-05-01 00:00:01", tz="Asia/Shanghai")
market_breath = {}
last_day_active_stocks = {}
for k, v in stock_candle_data_map.items():
    for i in range(len(v.index())-100, len(v.index())):
        close = v.close()[i]
        ma20 = v.raw_data['MA20'][i]
        current_day = v.index()[i]
        sector = stock_sector_map[k]
        current_day_data = market_breath.setdefault(str(current_day), {})

        value = current_day_data.setdefault(sector, {})
        all_value = current_day_data.setdefault('all', {})
        above = value.setdefault('above', 0)
        stocks_count = value.setdefault('all', 0)
        
        all_above = all_value.setdefault('above', 0)
        all_stocks_count = all_value.setdefault('all', 0)
        if close > ma20:
            if i == (len(v.index()) -1):
                last_day_sector_active_stock_list = last_day_active_stocks.setdefault(sector, [])
                last_day_sector_active_stock_list.append(v.name)
            current_day_data[sector]['above'] = above + 1
            current_day_data['all']['above'] = all_above + 1
        value['all'] = stocks_count + 1
        all_value['all'] = all_stocks_count + 1

print('----------------------')
print('\x1b[31m最后一天活跃公司\x1b[0m')
for k, v in last_day_active_stocks.items():
    print("\x1b[31m{}\x1b[0m:\n{}".format(k, ",".join(v)))
        
for k, v in market_breath.items():
    for name, value in v.items():
        value['percent'] = round(float(value['above'])*100/float(value['all']),2)

market_breath_show_data = []
for k, v in sorted(list(market_breath.items()), key=lambda x:x[0].lower(), reverse=True):
    result = [k[0:10]]
    for sector in sectors:
        result.append(str(v.get(sector, {}).get('percent', 0)))
    result.append(str(v.get('all',{}).get('percent', 0)))
    market_breath_show_data.append(result)
market_breath_column_name_list = ['日期']
market_breath_column_name_list.extend(sectors)
market_breath_column_name_list.append("ALL")
market_breath_data_frame=pd.DataFrame(market_breath_show_data, columns=market_breath_column_name_list)

result = None
if sectors_id == 'swsindex':
    result = market_breath_data_frame.style.applymap(market_breath_color, subset=market_breath_column_name_list[1:])
else:
    result = market_breath_data_frame.style.applymap(market_breath_color, subset=market_breath_column_name_list[1:]).set_table_styles(magnify())
result

swsindex_tag_tree.len(3959)
stock_sector_map len(3930)
300
全行业公司:
银行(25):
浦发银行,华夏银行,民生银行,招商银行,江苏银行,杭州银行,南京银行,渝农商行,兴业银行,北京银行,上海银行,农业银行,交通银行,工商银行,长沙银行,邮储银行,光大银行,成都银行,浙商银行,建设银行,中国银行,中信银行,平安银行,宁波银行,青农商行
交通运输(14):
白云机场,上海机场,上港集团,南方航空,东方航空,圆通速递,大秦铁路,春秋航空,中国国航,京沪高铁,招商轮船,中远海控,韵达股份,顺丰控股
钢铁(3):
包钢股份,宝钢股份,中信特钢
公用事业(8):
华能国际,华能水电,华电国际,国电电力,国投电力,长江电力,中国核电,中国广核
化工(11):
中国石化,中国巨石,安迪苏,万华化学,恒力石化,浙江龙盛,君正集团,恒逸石化,荣盛石化,龙蟒佰利,恩捷股份
非银金融(40):
中信证券,国投资本,国金证券,西南证券,五矿资本,中航资本,海通证券,中泰证券,东方证券,招商证券,中信建投,财通证券,天风证券,东兴证券,国泰君安,红塔证券,中国平安,中国人保,新华保险,兴业证券,东吴证券,中国太保,中国人寿,华泰证券,中银证券,光大证券,浙商证券,中国银河,方正证券,南京证券,申万宏源,天茂集团,国元证券,广发证券,长江证券,西部证券,国信证券,长城证券,华林证券,东方财富
机械设备(7):
三一重工,恒立液压,中国中车,中国通号,中微公司,中联重科,徐工机械
房地产(15):
保利地产,雅戈尔,新湖中宝,华夏幸福,金地集团,绿地控股,上海临港,新城控股,万科A,华侨城A,金科股份,阳光城,中南建设,招商蛇口,荣盛发展
通信(6):
中国联通,亨通光电,烽火通信,中天科技,中兴通讯,亿联网络
汽车(10):
宇通客车,上汽集团,广汇汽车,福耀玻璃,华域汽车,广汽集团,长城汽车,潍柴动力,长安汽车,比亚迪
建筑装饰(8):
葛洲坝,中国化学,中国铁建,中国中铁,中国中冶,中国建筑,中国电建,中国交建
医药生物(31):
同仁堂,天坛生物,复星医药,恒瑞医药,白云山,片仔癀,通策医疗,九州通,上海医药,甘李药业,药明康德,万泰生物,安图生物,云南白药,长春高新,华东医药,新和成,华兰生物,美年健康,上海莱士,科伦药业,康